In [1]:
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")

In [2]:
import torch
import os
print("Cuda is available : ", torch.cuda.is_available())
print("Cuda num : ", torch.cuda.device_count())  # gpu数量
print("Current cuda index : ", torch.cuda.current_device())
print("Current cuda name : ", torch.cuda.get_device_name(0))

Cuda is available :  True
Cuda num :  1
Current cuda index :  0
Current cuda name :  NVIDIA GeForce RTX 3090


In [3]:
from libs import *
dataPath = "../StockPriceForecast/dataSet/data/marketData/data.nc"
data_len = DataLength(dataPath)
print("All days : ", data_len)
input_days = 360
predict_days = 30
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[0, data_len - input_days - test_predict_len])
print("Train days : ", train_dataSet.data.shape)
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len])
print("Validation days ; ", validation_dataSet.data.shape)
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len])
print("Test days : ", test_dataSet.data.shape)

train_dataLoader = DataLoader(train_dataSet)
validation_dataLoader = DataLoader(validation_dataSet)
test_dataLoader = DataLoader(test_dataSet)

All days :  2994
Train days :  (2274, 1440, 10)
Validation days ;  (720, 1440, 10)
Test days :  (720, 1440, 10)


In [4]:
days_num = 100  ########################
train_dataSet = train_dataSet.isel(0, 390 + days_num -1)
train_dataLoader = DataLoader(train_dataSet)
print(len(train_dataLoader))

100


In [5]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
loss_function = "L1"  ####################
lr = 0.00005  ####################
model = ResNetSeq2SeqAttention(use_checkpoint=True,
                                resNet_layer_nums=[3,4,6,3],
                                output_size=1440,
                                seq2seq_hidden_size=4096,
                                seq2seq_num_layer=3)
# device_ids = [0, 1]
# model = torch.nn.DataParallel(model, device_ids=device_ids)
model.train()
device = torch.device("cuda:0")
model, attention_weight_seq, Loss =train_ResNetSeq2SeqAttention(model=model,
                    dataLoader=train_dataLoader, learning_rate=0.001,
                    device=device, num_epoch=1, init_weights=True, loss_function=loss_function, pre_days=30)

Loss function :  L1Loss()


  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.31 GiB (GPU 0; 23.70 GiB total capacity; 20.00 GiB already allocated; 282.56 MiB free; 21.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
save_path = "../StockPriceForecast/Models/test/spf_lr"+str(lr).split(".")[-1]+"_"+str(loss_function)+"_"+str(len(train_dataLoader))+".pth"
print(save_path)

In [ ]:
torch.save(model, save_path)

In [ ]:
model.eval()
model_outputs = []
target = []
attention_weights_seq = []
res_output = []
for data, tar in validation_dataLoader:
    data = data.reshape(360, 1, 120, 120).cuda().float()
    target = tar.detach().cpu()
    print(data.dtype)
    model_outputs, attention_weights_seq, _ = model(data, steps=30)
    print(model_outputs[0])
    break

In [ ]:
output = model_outputs.detach().cpu()
print(output.shape)
output = output.permute(2, 0, 1)
print(output.shape)
output = output.squeeze()
print(output.shape)

In [ ]:
target_data = target
print(target_data.shape)
target_data = target_data.permute(1, 0)
print(target_data.shape)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
show_index = 4
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = output[show_index].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = target_data[show_index].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [ ]:
print(len(attention_weights_seq))
print(attention_weights_seq[0].shape)
attention_weights = attention_weights_seq

In [ ]:
import numpy as np
for i in range(len(attention_weights_seq)):
    attention_weights[i].detach().cpu()
    attention_weights[i].require_grad=False
print(attention_weights[0].requires_grad)
attention_weights = np.array(attention_weights_seq)
print(attention_weights)

In [ ]:
print(attention_weights_seq[0])